# LBT PIT XML Editor
The LBT PIT proposal tool allows you to load targets from a csv file, but requires you to input manually the observing time for those targets.  This code takes the XML file the PIT tool creates and edits it to add to the XML file the observing times that are present, but unused, in the target csv files.

This code was copied from the Gemini PIT XML editor, the main difference being that proposal was for speckle observing and needed exposure times for two different seeing conditions, whereas this proposal is for spectrometry and only requires one exposure time.

Another difference is that this code creates a `<time>` node for each `<observation>` whereas the Gemini version finds an existing `<time>` node and edits its value.

In [1]:
from pathlib import Path

import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
proposal_cycle = "2026A"
base_dir = Path(f"../2025-10-24 LBT {proposal_cycle} Proposal")
filename = base_dir / f"LBT {proposal_cycle} Proposal template"
target_file = base_dir / f"LBT {proposal_cycle} Proposal Targets.csv"

# first, get observation times for all targets
target_list = pd.read_csv(target_file)
exposure_times = target_list.set_index("Name")["pepsi_exposure_time"].to_dict()

# next, get the mapping of the nickname used in xml file to the TIC name
tree = ET.parse(f"{filename}.xml")
root = tree.getroot()
target_map = {}
for target in root.findall("targets/sidereal"):
    designator = target.attrib["id"]
    name = target.find("name").text
    target_map[name] = designator

# finally, walk through each observation entry and add observing times to it
observations = root.find("observations")
for target_name in target_map.keys():
    observation = ET.SubElement(observations, "observation")
    observation.attrib = {"band": "Band 1", "enabled": "true", "target": target_map[target_name], "condition": "condition-0", "blueprint": "blueprint-0"}
    meta = ET.SubElement(observation, "meta")
    meta.attrib = {"ck": ""}
    visibility = ET.SubElement(meta, "visibility")
    visibility.text = "Good"
    aorefstars = ET.SubElement(meta, "aorefstars")
    aorefstars.text = "0"
    time_element = ET.SubElement(observation, "time")
    time_element.attrib = {"units": "hr", "bino": "false", "too": "None", "Run": ""}
    time_element.text = f"{exposure_times[target_name] / 3600:.3f}"
    tree.write(f"{filename}_times.xml")